In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython import display

In [71]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Resize(333),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5]),
])

In [72]:
train_data = torchvision.datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [85]:
class Xception(nn.Module):
    
    def __init__(self):
        super(Xception, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, 2, 0, bias=False)
            nn.BatchNorm2d(32)
            nn.ReLU(inplace=True)

            nn.Conv2d(32, 64, 3, bias=False)
            nn.BatchNorm2d(64)
            nn.ReLU(inplace=True)

            nn.Conv2d(32, 64, 3, bias=False)
            nn.BatchNorm2d(64)
            nn.ReLU(inplace=True)
        
            nn.Conv2d(64, 64, 2, 1, 1, 1,groups=64, bias=False)
            nn.Conv2d(64, 128, 1, 1, 0, 1, 1, bias=False)
            nn.BatchNorm2d(128)
            nn.ReLU(inplace=True)
        
            nn.Conv2d(128, 128, 2, 1, 1, 1,groups=128, bias=False)
            nn.Conv2d(128, 128, 1, 1, 0, 1, 1, bias=False)
            nn.BatchNorm2d(128)
            nn.ReLU(inplace=False)
        
            nn.Conv2d(128, 128, 2, 1, 1, 1,groups=128, bias=False)
            nn.Conv2d(128, 256, 1, 1, 0, 1, 1, bias=False)
            nn.BatchNorm2d(256)
            nn.ReLU(inplace=True)
        
            nn.Conv2d(256, 256, 2, 1, 1, 1,groups=256, bias=False)
            nn.Conv2d(256, 256, 1, 1, 0, 1, 1, bias=False)
            nn.BatchNorm2d(256)
            nn.ReLU(inplace=False)
        
        
#         self.convb31 = nn.Conv2d(256, 256, 2, 1, 1, 1,groups=256, bias=False)
#         self.convp31 = nn.Conv2d(256, 728, 1, 1, 0, 1, 1, bias=False)
#         self.bnb31 = nn.BatchNorm2d(728)
#         self.actb31 = nn.ReLU(inplace=True)       
        
#         self.convb32 = nn.Conv2d(728, 728, 2, 1, 1, 1,groups=728, bias=False)
#         self.convp32 = nn.Conv2d(728,728, 1, 1, 0, 1, 1, bias=False)
#         self.bnb32 = nn.BatchNorm2d(728)
        
        
#         self.actb41 = nn.ReLU(inplace=False)  
#         self.convb41 = nn.Conv2d(728, 728, 1, 1, 1, 1,groups=728, bias=False)
#         self.convp41 = nn.Conv2d(728, 728, 1, 1, 0, 1, 1, bias=False)
#         self.bnb41 = nn.BatchNorm2d(728)

#         self.actb42 = nn.ReLU(inplace=True)  
#         self.convb42 = nn.Conv2d(728, 728, 1, 1, 1, 1,groups=728, bias=False)
#         self.convp42 = nn.Conv2d(728, 728, 1, 1, 0, 1, 1, bias=False)
#         self.bnb42 = nn.BatchNorm2d(728)
 
#         self.actb43 = nn.ReLU(inplace=True)  
#         self.convb43 = nn.Conv2d(728, 728, 1, 1, 1, 1,groups=728, bias=False)
#         self.convp43 = nn.Conv2d(728, 728, 1, 1, 0, 1, 1, bias=False)
#         self.bnb43 = nn.BatchNorm2d(728)

#         self.actb121 = nn.ReLU(inplace=False)
#         self.convb121 = nn.Conv2d(728, 728, 2, 1, 1, 1,groups=728, bias=False)
#         self.convp121 = nn.Conv2d(728, 728, 1, 1, 0, 1, 1, bias=False)
#         self.bnb121 = nn.BatchNorm2d(728)
        
        
#         self.actb122 = nn.ReLU(inplace=True)
#         self.convb122 = nn.Conv2d(728, 728, 2, 1, 1, 1,groups=728, bias=False)
#         self.convp122 = nn.Conv2d(728, 1024, 1, 1, 0, 1, 1, bias=False)
#         self.bnb122 = nn.BatchNorm2d(1024)
        

#         self.conv3 = SeparableConv2d(1024, 1536, 3, 1, 1)
#         self.bn3 = nn.BatchNorm2d(1536)
#         self.act3 = nn.ReLU(inplace=True)

#         self.conv4 = SeparableConv2d(1536, 2048, 3, 1, 1)
#         self.bn4 = nn.BatchNorm2d(2048)
#         self.act4 = nn.ReLU(inplace=True))
        
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256* 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 10),
        )




    def forward_features(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act2(x)

        x = self.convp11(x)
        x = self.convb11(x)
        x = self.bnb11(x)
        x = self.actb11(x)
        
        x = self.convp12(x)
        x = self.convb12(x)
        x = self.bnb12(x)
        x = self.actb12(x)
        
        x = self.convp21(x)
        x = self.convb21(x)
        x = self.bnb21(x)
        x = self.actb21(x)
        
        x = self.convp22(x)
        x = self.convb22(x)
        x = self.bnb22(x)
        x = self.actb22(x)
        
#         x = self.convp31(x)
#         x = self.convb31(x)
#         x = self.bnb31(x)
#         x = self.actb31(x)
        
#         x = self.convp32(x)
#         x = self.convb32(x)
#         x = self.bnb32(x)
        
#         x = self.actb41(x)
#         x = self.convp41(x)
#         x = self.convb41(x)
#         x = self.bnb41(x)

#         x = self.actb42(x)
#         x = self.convp42(x)
#         x = self.convb42(x)
#         x = self.bnb42(x)
        
#         x = self.actb43(x)
#         x = self.convp43(x)
#         x = self.convb43(x)
#         x = self.bnb43(x)

#         x = self.actb121(x)
#         x = self.convp121(x)
#         x = self.convb121(x)
#         x = self.bnb121(x)

#         x = self.actb122(x)
#         x = self.convp122(x)
#         x = self.convb122(x)
#         x = self.bnb122(x)
        
        
#         x = self.conv3(x)
#         x = self.bn3(x)
#         x = self.act3(x)

#         x = self.conv4(x)
#         x = self.bn4(x)
#         x = self.act4(x)
        return x


SyntaxError: invalid syntax (<ipython-input-85-59c29a1cfcf9>, line 8)

In [80]:
from torch.utils.data.sampler import SubsetRandomSampler

num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

In [81]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

In [82]:
Xception = Xception()

(nn.Conv2d(1024, 1024, 3, 1, 1, 1,groups=1024, bias=False))
(nn.Conv2d(1024, 1536, 1, 1, 0, 1, 1, bias=False)
(nn.Conv2d(1536, 1536, 3, 1, 1, 1,groups=1536, bias=False))
(nn.Conv2d(1536, 2048, 1, 1, 0, 1, 1, bias=False)


In [83]:
import torch.optim as optim
import torch.nn as nn

#Loss
criterion = nn.CrossEntropyLoss()

#Optimizer(SGD)
optimizer = optim.SGD(Xception.parameters(), lr=0.001, momentum=0.9)
import torch.optim as optim
import torch.nn as nn

#Loss
criterion = nn.CrossEntropyLoss()

#Optimizer(SGD)
optimizer = optim.SGD(Xception.parameters(), lr=0.001, momentum=0.9)

In [84]:
for epoch in range(1): 
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0], data[1]


        optimizer.zero_grad()

        # forward + backward + optimize
        output = Xception(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training of Xception')

NotImplementedError: 